**程序初始模板**

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt



def main():
    pass




cv2.waitKey(0)
cv2.destroyAllWindows()

### 实战 1：切割出小猫
![](https://ws1.sinaimg.cn/large/acbcfa39gy1g0xxajfucaj20m80go0ud.jpg)

In [45]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


def nothing(x):
    pass

def sobel_edge(src, ddepth, ksize):
    """
    src: 灰度图像
    ddepth:  cv2.CV_64F
    ksize: eg. 3、5
    """
    sobelx = cv2.Sobel(src, ddepth, 1, 0, ksize)
    sobely = cv2.Sobel(src, ddepth, 0, 1, ksize)
    sobelx_abs = cv2.convertScaleAbs(sobelx)
    sobely_abs = cv2.convertScaleAbs(sobely)
    dst = cv2.addWeighted(sobelx_abs, 0.5, sobely_abs, 0.5, 0)
    cv2.imshow("sobel_edge", dst)
    return dst


def dilation_demo(img):
    """
    膨胀
    """
    # kernel = np.ones((5,5), np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilation  = cv2.dilate(img, kernel, iterations = 1)
    cv2.imshow("dilation", dilation)
    return dilation
    
def opening_demo(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    #opening = cv2.morphologyEx(img, 2, kernel) # 同上
    cv2.imshow("opening", opening)

    
    
def closing_demo(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    closing = cv2.morphologyEx(img, 3, kernel)
    cv2.imshow("closing", closing)
    
def gradient_demo(img):
    """
    img: 灰度图像
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    gradient  = cv2.morphologyEx(img, 4, kernel)
    
    cv2.imshow("gradient ", gradient)
    return gradient
    
    
def cat_demo(src):
    gray = cv2.cvtColor(src, 6)
    #dilation = dilation_demo(gray)
    #edge = sobel_edge(gray, cv2.CV_64F, 3)
    canny = cv2.Canny(gray, 27, 80)
    _, thresh = cv2.threshold(canny, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cv2.imshow("thresh", thresh)
    dilation = dilation_demo(thresh)
    gradient_demo(dilation)
    closing_demo(dilation)
    

def main():
    img = cv2.imread("./sample_img/cat.jpg")
    #img = img[235:516, 120:685, :]
    blur = cv2.GaussianBlur(img, (3, 3), 0) 
    cv2.imshow("src", img)
    cat_demo(blur)



main()
cv2.waitKey(0)
cv2.destroyAllWindows()

### 车牌定位

#### 基本知识（场景分析，分析研究对象的常识，基本特征，方便进一步分析）——常识角度、图像处理角度

我国的汽车牌照一般由七个字符和一个点组成，车牌字符的高度和宽度是固定的，分别为90mm和45mm，七个字符之间的距离也是固定的12mm，点分割符的直径是10mm，当然字符间的差异可能会引起字符间的距离变化。在民用车牌中，字符的排列位置遵循以下规律：第一个字符通常是我国各省区的简称，用汉字表示；第二个字符通常是发证机关的代码号，最后五个字符由英文字母和数字组合而成，字母是二十四个大写字母（除去I和O这两个字母）的组合，数字用＂０-９＂之间的数字表示。

从图像处理角度看，汽车牌照有以下几个特征：
- 第一个特征是是车牌的几何特征，即车牌形状统一为长宽高固定的矩形；
- 第二个特征是车牌的灰度分布呈现出连续的波谷－波峰－波谷分布，这是因为我国车牌颜色单一，字符直线排列；
- 第三个特征是车牌直方图呈现出双峰状的特点，即车牌直方图中可以看到双个波峰；
- 第四个特征是车牌具有强边缘信息，这是因为车牌的字符相对集中在车牌的中心，而车牌边缘无字符，因此车牌的边缘信息感较强；
- 第五个特征是车牌的字符颜色和车牌背景颜色对比鲜明。目前，我国国内的车牌大致可分为蓝底白字和黄底黑字，特殊用车采用白底黑字或黑底白字，有时辅以红色字体等。为了简化处理，本次学习中只考虑蓝底白字的车牌。


#### 图像处理过程
- 图像大小预处理
- 灰度处理
- 图像降噪
- 形态学处理
- 阈值处理，使用轨迹条进行调试
- 边缘检测



In [188]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


def resize_img(src, MAX_HIGH=1000, rate=1):
    """
    图像大小处理，防止图像太大
    img: 输入图像
    输出调整后的图像

    ———————— 基本原理  ——————————
    Opencv 中的 resize 函数
    resize(src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst
        src: 输入的图像
        dsize: 输出图像的大小，设置为 None 则由 fx, fy 决定
        fx: 宽度的缩放比例
        fy: 高度的缩放比例
        interpolation：插值方法
        INTER_NEAREST - 最近邻插值 (速度最快)
        INTER_LINEAR - 双线性插值（默认） 放大的效果还 OK ，速度较快
        INTER_AREA - 使用像素区域关系重新采样。它可能是图像抽取的首选方法，因为它可以提供无莫尔条纹的结果。但是当图像被缩放时，它类似于 INTER_NEAREST 方法。 缩小图像效果最好的方法
        INTER_CUBIC - 4x4 像素邻域上的双三次插值 放大的效果最好（慢）
        INTER_LANCZOS4 - 8x8 像素邻域的 Lanczos 插值
        dst: 输出图像
    """
    img = src
    rows, cols = img.shape[0:2]  # 获取图像宽度和高度
    if cols > MAX_HIGH:  # 高度超过阈值
        change_rate = MAX_HIGH / cols  # 变换的比例
        img = cv2.resize(img, (int(rows*change_rate*rate),
                               int(cols*change_rate*rate)), interpolation=cv2.INTER_AREA)
    return img


def nothing(x):
    pass


def trackbar_demo():
    """
    轨迹条函数，不调用，只是写下来，用的时候方便
    """
    cv2.namedWindow("dst")
    cv2.createTrackbar("low", "dst", 0, 255, nothing)
    low = cv2.getTrackbarPos("low", "dst")


def del_noise(img, ksize=3):
    """
    输入图像和内核大小，内核只能是奇数
    输滤波后的图像
    —————————基本原理—————————
    高斯滤波
    GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
        src: 源图片
        ksize: 内核大小,。其中 ksize.width 和 ksize.height 可以不同，但他们都必须为正数和奇数。或者，它们可以是零的，它们都是由 sigma 计算而来。
        sigmaX: 表示高斯核函数在 X 方向的的标准偏差。
        dst：目标输出
        sigmaY：表示高斯核函数在 Y 方向的的标准偏差。若 sigmaY 为零，就将它设为 sigmaX ，如果 sigmaX 和 sigmaY 都是 0，那么就由 ksize.width 和ksize.height 计算出来。
        borderType: 边缘处理参数
    """
    blur = cv2.GaussianBlur(img, (ksize, ksize), 0, 0, cv2.BORDER_DEFAULT)
    cv2.imshow("blur", blur)
    return blur


def erosion_demo(img, ksize=5):
    """
    腐蚀操作:膨胀是取每一个位置邻域内的最小值

    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    erosion = cv2.erode(img, kernel, iterations=1)
    cv2.imshow("erosion", erosion)
    return erosion


def dilation_demo(img, ksize=5):
    """
    膨胀操作:膨胀是取每一个位置邻域内的最大值
    —————————基本原理—————————
    膨胀后的输出图像的总体亮度的平均值比起原图会有所上升，而图像中较亮物体的尺寸会变大；
    相反，较暗物体的尺寸会减小，甚至消失。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    dilation = cv2.dilate(img, kernel, iterations=1)
    cv2.imshow("dilation", dilation)
    return dilation


def opening_demo(img, ksize=5):
    """
    开运算
    先进行 erode 再进行 dilate 的过程就是开运算，
    它具有消除亮度较高的细小区域、在纤细点处分离物体，
    对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    #opening = cv2.addWeighted(img, 1, opening, -1, 0)
    cv2.imshow("opening", opening)
    return opening


def closing_demo(img, ksize=5):        
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return closing
    cv2.imshow("closing", closing)

def threshold_demo(img, thresh1=127, thresh2=255):
    """
    阈值处理
    """
    _, thresh = cv2.threshold(img, thresh1, thresh1, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # print(thresh1, thresh2)
    cv2.imshow("thresh", thresh)
    return thresh

def hsv_trace(img):
    """
    颜色跟踪，img 三通道图像
    """
    # BGR -> HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 在 HSV 中定义蓝色的范围
    lower_blue = np.array([100, 43, 46], dtype=np.uint8)
    upper_blue = np.array([124, 255, 255], dtype=np.uint8)

    # 将 HSV 图像的阈值设置为只获取蓝色
    mask = cv2.inRange(hsv, lower_blue, upper_blue) 
    # 把像素值在 lower_blue 和 upper_blue 之间的像素置 255(白)，之外的置 0（黑）

    res = cv2.bitwise_and(img, img, mask=mask) # 白留黑除
    cv2.imshow("hsv", hsv) # 
    cv2.imshow("mask", mask)
    cv2.imshow("res", res)

def main():


    # 把图像读取出来
    img = cv2.imread("./sample_img/car_num_4.jpg")
    img = resize_img(src=img)
    #cv2.imshow("src", img)
    
    # 灰度化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("gray", gray)

    # 图像滤波，降噪
    blur = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)
    cv2.imshow("blur", blur)
    
    # 形态学处理，开运算
    # 它具有消除亮度较高的细小区域、在纤细点处分离物体，
    #      对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
    opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel)
    cv2.imshow("opening", opening)
    opening = cv2.addWeighted(blur, 1, opening, -1, 0) # 滤波后图像减去开运算后的图像得到
    cv2.imshow("opening_2", opening)
      
    # 寻找边缘
    _, thresh = cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    canny_img = cv2.Canny(thresh, 10, 200)
    cv2.imshow("canny_img", canny_img)
    # --------------------以上的效果不错，重点优化下面---------------------------------------
    
    
    """    # 闭运算和开运算之后，使边缘成为一个整体
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
    closing = cv2.morphologyEx(canny_img, cv2.MORPH_CLOSE, kernel)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    cv2.imshow("closing", closing)
    cv2.imshow("opening_3", opening)"""
    
    """
    # 查找图像边缘整体形成的矩形区域，可能有很多，车牌就在其中一个
    _, contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    temp_contours = []
    Min_Area = 2000
    for contour in contours:
        if cv2.contourArea(contour) > Min_Area:
            temp_contours.append(contour)
    
    Min_Area = 6000
    for contour in contours:
        if cv2.contourArea(contour) > Min_Area:
            temp_contours.append(contour)

    car_plate = []
    for temp_contour in temp_contours:
        # 返回 ((center_x, center_y), h, w, degree)
        rect = cv2.minAreaRect(temp_contour)
        rect_h, rect_w = rect[1]
        if rect_w > rect_h:  # 保持高永远是最大的
            rect_h, rect_w = rect_w, rect_h
        rect_rate = rect_h / rect_w
        if rect_rate > 2 and rect_rate < 5.5:
            car_plate.append(temp_contour)

        box = cv2.boxPoints(rect)
        box = np.int0(box)
        #print(box)
        img = cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

    cv2.imshow("src", img)
    """

    

    
    
    
main()


cv2.waitKey(0)
cv2.destroyAllWindows()

[参考](https://blog.csdn.net/weixin_41695564/article/details/79712393)

In [184]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


def resize_img(src, MAX_HIGH=1000, rate=1):
    """
    图像大小处理，防止图像太大
    img: 输入图像
    输出调整后的图像

    ———————— 基本原理  ——————————
    Opencv 中的 resize 函数
    resize(src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst
        src: 输入的图像
        dsize: 输出图像的大小，设置为 None 则由 fx, fy 决定
        fx: 宽度的缩放比例
        fy: 高度的缩放比例
        interpolation：插值方法
        INTER_NEAREST - 最近邻插值 (速度最快)
        INTER_LINEAR - 双线性插值（默认） 放大的效果还 OK ，速度较快
        INTER_AREA - 使用像素区域关系重新采样。它可能是图像抽取的首选方法，因为它可以提供无莫尔条纹的结果。但是当图像被缩放时，它类似于 INTER_NEAREST 方法。 缩小图像效果最好的方法
        INTER_CUBIC - 4x4 像素邻域上的双三次插值 放大的效果最好（慢）
        INTER_LANCZOS4 - 8x8 像素邻域的 Lanczos 插值
        dst: 输出图像
    """
    img = src
    rows, cols = img.shape[0:2]  # 获取图像宽度和高度
    if cols > MAX_HIGH:  # 高度超过阈值
        change_rate = MAX_HIGH / cols  # 变换的比例
        img = cv2.resize(img, (int(rows*change_rate*rate),
                               int(cols*change_rate*rate)), interpolation=cv2.INTER_AREA)
    return img


def nothing(x):
    pass


def trackbar_demo():
    """
    轨迹条函数，不调用，只是写下来，用的时候方便
    """
    cv2.namedWindow("dst")
    cv2.createTrackbar("low", "dst", 0, 255, nothing)
    low = cv2.getTrackbarPos("low", "dst")


def del_noise(img, ksize=3):
    """
    输入图像和内核大小，内核只能是奇数
    输滤波后的图像
    —————————基本原理—————————
    高斯滤波
    GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
        src: 源图片
        ksize: 内核大小,。其中 ksize.width 和 ksize.height 可以不同，但他们都必须为正数和奇数。或者，它们可以是零的，它们都是由 sigma 计算而来。
        sigmaX: 表示高斯核函数在 X 方向的的标准偏差。
        dst：目标输出
        sigmaY：表示高斯核函数在 Y 方向的的标准偏差。若 sigmaY 为零，就将它设为 sigmaX ，如果 sigmaX 和 sigmaY 都是 0，那么就由 ksize.width 和ksize.height 计算出来。
        borderType: 边缘处理参数
    """
    blur = cv2.GaussianBlur(img, (ksize, ksize), 0, 0, cv2.BORDER_DEFAULT)
    cv2.imshow("blur", blur)
    return blur


def erosion_demo(img, ksize=5):
    """
    腐蚀操作:膨胀是取每一个位置邻域内的最小值

    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    erosion = cv2.erode(img, kernel, iterations=1)
    cv2.imshow("erosion", erosion)
    return erosion


def dilation_demo(img, ksize=5):
    """
    膨胀操作:膨胀是取每一个位置邻域内的最大值
    —————————基本原理—————————
    膨胀后的输出图像的总体亮度的平均值比起原图会有所上升，而图像中较亮物体的尺寸会变大；
    相反，较暗物体的尺寸会减小，甚至消失。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    dilation = cv2.dilate(img, kernel, iterations=1)
    cv2.imshow("dilation", dilation)
    return dilation


def opening_demo(img, ksize=5):
    """
    开运算
    先进行 erode 再进行 dilate 的过程就是开运算，
    它具有消除亮度较高的细小区域、在纤细点处分离物体，
    对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    #opening = cv2.addWeighted(img, 1, opening, -1, 0)
    cv2.imshow("opening", opening)
    return opening


def closing_demo(img, ksize=5):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return closing
    cv2.imshow("closing", closing)


def threshold_demo(img, thresh1=127, thresh2=255):
    """
    阈值处理
    """
    _, thresh = cv2.threshold(img, thresh1, thresh1,
                              cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # print(thresh1, thresh2)
    cv2.imshow("thresh", thresh)
    return thresh

def hsv_trace(img):
    """
    颜色跟踪，img 三通道图像
    """
    # BGR -> HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 在 HSV 中定义蓝色的范围
    lower_blue = np.array([100, 43, 46], dtype=np.uint8)
    upper_blue = np.array([124, 255, 255], dtype=np.uint8)

    # 将 HSV 图像的阈值设置为只获取蓝色
    mask = cv2.inRange(hsv, lower_blue, upper_blue) 
    # 把像素值在 lower_blue 和 upper_blue 之间的像素置 255(白)，之外的置 0（黑）

    res = cv2.bitwise_and(img, img, mask=mask) # 白留黑除
    
    cv2.imshow("res", res)
    return res
    #cv2.imshow("hsv", hsv) # 
    #cv2.imshow("mask", mask)
    

def main():
    # 把图像读取出来
    img = cv2.imread("./sample_img/car_num_4.jpg")
    img = resize_img(src=img, rate=0.5)
    #cv2.imshow("src", img)
    hsv = hsv_trace(img)
    # 灰度化
    gray = cv2.cvtColor(hsv, cv2.COLOR_BGR2GRAY)
    cv2.imshow("gray", gray)

    # 图像滤波，降噪
    blur = cv2.GaussianBlur(gray, (5, 5), 0, 0, cv2.BORDER_DEFAULT)
    cv2.imshow("blur", blur)

    # 形态学处理，开运算
    # 它具有消除亮度较高的细小区域、在纤细点处分离物体，
    #      对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel)
    cv2.imshow("opening", opening)
    opening = cv2.addWeighted(blur, 1, opening, -1, 0)  # 滤波后图像减去开运算后的图像得到
    cv2.imshow("opening_2", opening)

    # 寻找边缘
    _, thresh = cv2.threshold(
        opening, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    canny_img = cv2.Canny(thresh, 100, 200)
    cv2.imshow("canny_img", canny_img)

    # 闭运算和开运算之后，使边缘成为一个整体
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    closing = cv2.morphologyEx(canny_img, cv2.MORPH_CLOSE, kernel)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    cv2.imshow("closing", closing)
    cv2.imshow("opening_3", opening)

    # 查找图像边缘整体形成的矩形区域，可能有很多，车牌就在其中一个
    _, contours, hierarchy = cv2.findContours(
        opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    temp_contours = []
    Min_Area = 6000
    for contour in contours:
        if cv2.contourArea(contour) > Min_Area:
            temp_contours.append(contour)

    car_plate = []
    for temp_contour in temp_contours:
        # 返回 ((center_x, center_y), h, w, degree)
        rect = cv2.minAreaRect(temp_contour)
        rect_h, rect_w = rect[1]
        if rect_w > rect_h:  # 保持高永远是最大的
            rect_h, rect_w = rect_w, rect_h
        rect_rate = rect_h / rect_w
        if rect_rate > 2 and rect_rate < 5.5:
            car_plate.append(temp_contour)

        box = cv2.boxPoints(rect)
        box = np.int0(box)
        #print(box)
        img = cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

    cv2.imshow("src", img)



main()


cv2.waitKey(0)
cv2.destroyAllWindows()

In [190]:
import cv2

# 1、读取图像，并把图像转换为灰度图像并显示
img = cv2.imread("./sample_img/car_num_2.jpg")  # 读取图片
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   # 转换了灰度化
cv2.imshow('gray', img_gray)  # 显示图片


# 2、将灰度图像二值化，设定阈值是100
img_thre = img_gray
cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV, img_thre)
cv2.imshow('threshold', img_thre)


# 3、保存黑白图片
#cv2.imwrite('thre_res.png', img_thre)

# 4、分割字符
white = []  # 记录每一列的白色像素总和
black = []  # ..........黑色.......
height = img_thre.shape[0]
width = img_thre.shape[1]
white_max = 0
black_max = 0
# 计算每一列的黑白色像素总和
for i in range(width):
    s = 0  # 这一列白色总数
    t = 0  # 这一列黑色总数
    for j in range(height):
        if img_thre[j][i] == 255:
            s += 1
        if img_thre[j][i] == 0:
            t += 1
    white_max = max(white_max, s)
    black_max = max(black_max, t)
    white.append(s)
    black.append(t)
    # print(s)
    # print(t)

arg = False  # False表示白底黑字；True表示黑底白字
if black_max > white_max:
    arg = True

# 分割图像


def find_end(start_):
    end_ = start_+1
    for m in range(start_+1, width-1):
        # 0.95这个参数请多调整，对应下面的0.05
        if (black[m] if arg else white[m]) > (0.95 * black_max if arg else 0.95 * white_max):
            end_ = m
            break
    return end_


n = 1
start = 1
end = 2
while n < width-2:
    n += 1
    if (white[n] if arg else black[n]) > (0.05 * white_max if arg else 0.05 * black_max):
        # 上面这些判断用来辨别是白底黑字还是黑底白字
        # 0.05这个参数请多调整，对应上面的0.95
        start = n
        end = find_end(start)
        n = end
        if end-start > 5:
            cj = img_thre[1:height, start:end]
            cv2.imshow('caijian', cj)
            
            
cv2.waitKey(0)
cv2.destroyAllWindows()

##### 效果还可以的 v1

In [242]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os


def resize_img(src, MAX_HIGH=1000, rate=1):
    """
    图像大小处理，防止图像太大
    img: 输入图像
    输出调整后的图像

    ———————— 基本原理  ——————————
    Opencv 中的 resize 函数
    resize(src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst
        src: 输入的图像
        dsize: 输出图像的大小，设置为 None 则由 fx, fy 决定
        fx: 宽度的缩放比例
        fy: 高度的缩放比例
        interpolation：插值方法
        INTER_NEAREST - 最近邻插值 (速度最快)
        INTER_LINEAR - 双线性插值（默认） 放大的效果还 OK ，速度较快
        INTER_AREA - 使用像素区域关系重新采样。它可能是图像抽取的首选方法，因为它可以提供无莫尔条纹的结果。但是当图像被缩放时，它类似于 INTER_NEAREST 方法。 缩小图像效果最好的方法
        INTER_CUBIC - 4x4 像素邻域上的双三次插值 放大的效果最好（慢）
        INTER_LANCZOS4 - 8x8 像素邻域的 Lanczos 插值
        dst: 输出图像
    """
    img = src
    rows, cols = img.shape[0:2]  # 获取图像宽度和高度
    if cols > MAX_HIGH:  # 高度超过阈值
        change_rate = MAX_HIGH / cols  # 变换的比例
        img = cv2.resize(img, (int(rows*change_rate*rate),
                               int(cols*change_rate*rate)), interpolation=cv2.INTER_AREA)
    return img


def nothing(x):
    pass


def trackbar_demo():
    """
    轨迹条函数，不调用，只是写下来，用的时候方便
    """
    cv2.namedWindow("dst")
    cv2.createTrackbar("low", "dst", 0, 255, nothing)
    low = cv2.getTrackbarPos("low", "dst")


def del_noise(img, ksize=3):
    """
    输入图像和内核大小，内核只能是奇数
    输滤波后的图像
    —————————基本原理—————————
    高斯滤波
    GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
        src: 源图片
        ksize: 内核大小,。其中 ksize.width 和 ksize.height 可以不同，但他们都必须为正数和奇数。或者，它们可以是零的，它们都是由 sigma 计算而来。
        sigmaX: 表示高斯核函数在 X 方向的的标准偏差。
        dst：目标输出
        sigmaY：表示高斯核函数在 Y 方向的的标准偏差。若 sigmaY 为零，就将它设为 sigmaX ，如果 sigmaX 和 sigmaY 都是 0，那么就由 ksize.width 和ksize.height 计算出来。
        borderType: 边缘处理参数
    """
    blur = cv2.GaussianBlur(img, (ksize, ksize), 0, 0, cv2.BORDER_DEFAULT)
    cv2.imshow("blur", blur)
    return blur


def erosion_demo(img, ksize=5):
    """
    腐蚀操作:膨胀是取每一个位置邻域内的最小值

    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    erosion = cv2.erode(img, kernel, iterations=1)
    cv2.imshow("erosion", erosion)
    return erosion


def dilation_demo(img, ksize=5):
    """
    膨胀操作:膨胀是取每一个位置邻域内的最大值
    —————————基本原理—————————
    膨胀后的输出图像的总体亮度的平均值比起原图会有所上升，而图像中较亮物体的尺寸会变大；
    相反，较暗物体的尺寸会减小，甚至消失。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    dilation = cv2.dilate(img, kernel, iterations=1)
    cv2.imshow("dilation", dilation)
    return dilation


def opening_demo(img, ksize=5):
    """
    开运算
    先进行 erode 再进行 dilate 的过程就是开运算，
    它具有消除亮度较高的细小区域、在纤细点处分离物体，
    对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    #opening = cv2.addWeighted(img, 1, opening, -1, 0)
    cv2.imshow("opening", opening)
    return opening


def closing_demo(img, ksize=5):        
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return closing
    cv2.imshow("closing", closing)

def threshold_demo(img, thresh1=127, thresh2=255):
    """
    阈值处理
    """
    _, thresh = cv2.threshold(img, thresh1, thresh1, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # print(thresh1, thresh2)
    cv2.imshow("thresh", thresh)
    return thresh

def hsv_trace(img):
    """
    颜色跟踪，img 三通道图像
    """
    # BGR -> HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 在 HSV 中定义蓝色的范围
    lower_blue = np.array([100, 43, 46], dtype=np.uint8)
    upper_blue = np.array([124, 255, 255], dtype=np.uint8)

    # 将 HSV 图像的阈值设置为只获取蓝色
    mask = cv2.inRange(hsv, lower_blue, upper_blue) 
    # 把像素值在 lower_blue 和 upper_blue 之间的像素置 255(白)，之外的置 0（黑）

    res = cv2.bitwise_and(img, img, mask=mask) # 白留黑除
    cv2.imshow("hsv", hsv) # 
    cv2.imshow("mask", mask)
    cv2.imshow("res", res)

def main(img):


    # 把图像读取出来
    #img = cv2.imread("./sample_img/car_num/car_num.jpg")
    img = resize_img(src=img, rate=0.5)
    #cv2.imshow("src", img)
    
    # 灰度化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("gray", gray)

    # 图像滤波，降噪
    blur = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)
    #cv2.imshow("blur", blur)
    
    # 形态学处理，开运算
    # 它具有消除亮度较高的细小区域、在纤细点处分离物体，
    #      对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
    opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel)
    #cv2.imshow("opening", opening)
    opening = cv2.addWeighted(blur, 1, opening, -1, 0) # 滤波后图像减去开运算后的图像得到
    #cv2.imshow("opening_2", opening)
      
    # 寻找边缘
    _, thresh = cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    canny_img = cv2.Canny(thresh, 10, 200)
    #cv2.imshow("canny_img", canny_img)
    # --------------------以上的效果不错，重点优化下面---------------------------------------
    
    
     # 闭运算和开运算之后，使边缘成为一个整体
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (26, 26))
    closing = cv2.morphologyEx(canny_img, cv2.MORPH_CLOSE, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (26, 26))
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    #cv2.imshow("closing", closing)
    #cv2.imshow("opening_3", opening)
    

    # 查找图像边缘整体形成的矩形区域，可能有很多，车牌就在其中一个
    _, contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    temp_contours = []
    Min_Area = 15000
    for contour in contours:
        #area = cv2.contourArea(contour)
        #print(area)
        if cv2.contourArea(contour) > Min_Area:
            temp_contours.append(contour)
    

    # 根据车牌的物理特征找到它
    car_plate = []
    for temp_contour in temp_contours:
        # 返回 ((center_x, center_y), h, w, degree)
        rect = cv2.minAreaRect(temp_contour)
        rect_h, rect_w = rect[1]
        if rect_w > rect_h:  # 保持高永远是最大的
            rect_h, rect_w = rect_w, rect_h
        rect_rate = rect_h / rect_w
        if rect_rate > 2 and rect_rate < 5.5:
            car_plate.append(temp_contour)

        box = cv2.boxPoints(rect)
        box = np.int0(box)
        #print(box)
        img = cv2.drawContours(img, [box], 0, (0, 0, 255), 2)
    
    
    cv2.imshow("src"+str(np.random.randint(10)), img)
    
    

    

    
file = os.listdir("./sample_img/car_num/")

for i in range(len(file)):
    img = cv2.imread("./sample_img/car_num/"+file[i])    
    main(img)


cv2.waitKey(0)
cv2.destroyAllWindows()

##### 效果还可以的 v2

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os


def resize_img(src, MAX_HIGH=1000, rate=1):
    """
    图像大小处理，防止图像太大
    img: 输入图像
    输出调整后的图像

    ———————— 基本原理  ——————————
    Opencv 中的 resize 函数
    resize(src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst
        src: 输入的图像
        dsize: 输出图像的大小，设置为 None 则由 fx, fy 决定
        fx: 宽度的缩放比例
        fy: 高度的缩放比例
        interpolation：插值方法
        INTER_NEAREST - 最近邻插值 (速度最快)
        INTER_LINEAR - 双线性插值（默认） 放大的效果还 OK ，速度较快
        INTER_AREA - 使用像素区域关系重新采样。它可能是图像抽取的首选方法，因为它可以提供无莫尔条纹的结果。但是当图像被缩放时，它类似于 INTER_NEAREST 方法。 缩小图像效果最好的方法
        INTER_CUBIC - 4x4 像素邻域上的双三次插值 放大的效果最好（慢）
        INTER_LANCZOS4 - 8x8 像素邻域的 Lanczos 插值
        dst: 输出图像
    """
    img = src
    rows, cols = img.shape[0:2]  # 获取图像宽度和高度
    if cols > MAX_HIGH:  # 高度超过阈值
        change_rate = MAX_HIGH / cols  # 变换的比例
        img = cv2.resize(img, (int(rows*change_rate*rate),
                               int(cols*change_rate*rate)), interpolation=cv2.INTER_AREA)
    return img


def nothing(x):
    pass


def trackbar_demo():
    """
    轨迹条函数，不调用，只是写下来，用的时候方便
    """
    cv2.namedWindow("dst")
    cv2.createTrackbar("low", "dst", 0, 255, nothing)
    low = cv2.getTrackbarPos("low", "dst")


def del_noise(img, ksize=3):
    """
    输入图像和内核大小，内核只能是奇数
    输滤波后的图像
    —————————基本原理—————————
    高斯滤波
    GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
        src: 源图片
        ksize: 内核大小,。其中 ksize.width 和 ksize.height 可以不同，但他们都必须为正数和奇数。或者，它们可以是零的，它们都是由 sigma 计算而来。
        sigmaX: 表示高斯核函数在 X 方向的的标准偏差。
        dst：目标输出
        sigmaY：表示高斯核函数在 Y 方向的的标准偏差。若 sigmaY 为零，就将它设为 sigmaX ，如果 sigmaX 和 sigmaY 都是 0，那么就由 ksize.width 和ksize.height 计算出来。
        borderType: 边缘处理参数
    """
    blur = cv2.GaussianBlur(img, (ksize, ksize), 0, 0, cv2.BORDER_DEFAULT)
    cv2.imshow("blur", blur)
    return blur


def erosion_demo(img, ksize=5):
    """
    腐蚀操作:膨胀是取每一个位置邻域内的最小值

    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    erosion = cv2.erode(img, kernel, iterations=1)
    cv2.imshow("erosion", erosion)
    return erosion


def dilation_demo(img, ksize=5):
    """
    膨胀操作:膨胀是取每一个位置邻域内的最大值
    —————————基本原理—————————
    膨胀后的输出图像的总体亮度的平均值比起原图会有所上升，而图像中较亮物体的尺寸会变大；
    相反，较暗物体的尺寸会减小，甚至消失。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    dilation = cv2.dilate(img, kernel, iterations=1)
    cv2.imshow("dilation", dilation)
    return dilation


def opening_demo(img, ksize=5):
    """
    开运算
    先进行 erode 再进行 dilate 的过程就是开运算，
    它具有消除亮度较高的细小区域、在纤细点处分离物体，
    对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    #opening = cv2.addWeighted(img, 1, opening, -1, 0)
    cv2.imshow("opening", opening)
    return opening


def closing_demo(img, ksize=5):        
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (ksize, ksize))
    closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return closing
    cv2.imshow("closing", closing)

def threshold_demo(img, thresh1=127, thresh2=255):
    """
    阈值处理
    """
    _, thresh = cv2.threshold(img, thresh1, thresh1, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # print(thresh1, thresh2)
    cv2.imshow("thresh", thresh)
    return thresh

def hsv_trace(img):
    """
    颜色跟踪，img 三通道图像
    """
    # BGR -> HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 在 HSV 中定义蓝色的范围
    lower_blue = np.array([100, 43, 46], dtype=np.uint8)
    upper_blue = np.array([124, 255, 255], dtype=np.uint8)

    # 将 HSV 图像的阈值设置为只获取蓝色
    mask = cv2.inRange(hsv, lower_blue, upper_blue) 
    # 把像素值在 lower_blue 和 upper_blue 之间的像素置 255(白)，之外的置 0（黑）

    res = cv2.bitwise_and(img, img, mask=mask) # 白留黑除
    cv2.imshow("hsv", hsv) # 
    cv2.imshow("mask", mask)
    cv2.imshow("res", res)

    
def main(img):


    # 把图像读取出来
    #img = cv2.imread("./sample_img/car_num/car_num.jpg")
    img = resize_img(src=img, rate=0.5)
    #cv2.imshow("src", img)
    
    # 灰度化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("gray", gray)

    # 图像滤波，降噪
    blur = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)
    #cv2.imshow("blur", blur)
    
    # 形态学处理，开运算
    # 它具有消除亮度较高的细小区域、在纤细点处分离物体，
    #      对于较大物体，可以在不明显改变其面积的情况下平滑其边界等作用。 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
    opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel)
    #cv2.imshow("opening", opening)
    opening = cv2.addWeighted(blur, 1, opening, -1, 0) # 滤波后图像减去开运算后的图像得到
    #cv2.imshow("opening_2", opening)
      
    # 寻找边缘
    _, thresh = cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    canny_img = cv2.Canny(thresh, 10, 200)
    #cv2.imshow("canny_img", canny_img)
    # --------------------以上的效果不错，重点优化下面---------------------------------------
    
    
     # 闭运算和开运算之后，使边缘成为一个整体
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (26, 26))
    closing = cv2.morphologyEx(canny_img, cv2.MORPH_CLOSE, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (26, 26))
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    #cv2.imshow("closing", closing)
    #cv2.imshow("opening_3", opening)
    

    # 查找图像边缘整体形成的矩形区域，可能有很多，车牌就在其中一个
    _, contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    temp_contours = []
    Min_Area = 15000
    for contour in contours:
        #area = cv2.contourArea(contour)
        #print(area)
        if cv2.contourArea(contour) > Min_Area:
            temp_contours.append(contour)
    

    # 根据车牌的物理特征找到它
    car_plate = []
    for temp_contour in temp_contours:
        # 返回 ((center_x, center_y), h, w, degree)
        rect = cv2.minAreaRect(temp_contour)
        rect_h, rect_w = rect[1]
        if rect_w > rect_h:  # 保持高永远是最大的
            rect_h, rect_w = rect_w, rect_h
        rect_rate = rect_h / rect_w
        if rect_rate > 2 and rect_rate < 5.5:
            car_plate.append(temp_contour)

        box = cv2.boxPoints(rect)
        box = np.int0(box)
        #print(box)
        img = cv2.drawContours(img, [box], 0, (0, 0, 255), 2)
    
    
    cv2.imshow("src"+str(np.random.randint(10)), img)
    
    

    

    
file = os.listdir("./sample_img/car_num/")

for i in range(len(file)):
    img = cv2.imread("./sample_img/car_num/"+file[i])    
    main(img)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# -*- coding: utf-8 -*-
# @Author: hyzhangyong
# @Date:   2016-06-23 16:21:54
# @Last Modified by:   hyzhangyong
# @Last Modified time: 2016-06-24 00:00:47
import sys
import cv2
import numpy as np
from PIL import Image

# 旋转图片
def rotate(img,path):
        im1 = Image.open(img)
        im2 = im1.rotate(180)
        im2.save(path)

def preprocess(gray):
        # # 直方图均衡化
        equ = cv2.equalizeHist(gray)
        # 高斯平滑
        gaussian = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)
        # 中值滤波
        median = cv2.medianBlur(gaussian, 5)
        # Sobel算子，X方向求梯度
        sobel = cv2.Sobel(median, cv2.CV_8U, 1, 0, ksize = 3)
        # 二值化
        ret, binary = cv2.threshold(sobel, 170, 255, cv2.THRESH_BINARY)
        # 膨胀和腐蚀操作的核函数
        element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
        element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 7))
        # 膨胀一次，让轮廓突出
        dilation = cv2.dilate(binary, element2, iterations = 1)
        # 腐蚀一次，去掉细节
        erosion = cv2.erode(dilation, element1, iterations = 1)
        # 再次膨胀，让轮廓明显一些
        dilation2 = cv2.dilate(erosion, element2,iterations = 3)
        # cv2.imshow('dilation2',dilation2)
        # cv2.waitKey(0)
        return dilation2

def findPlateNumberRegion(img):
        region = []
        # 查找轮廓
        contours,hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # 筛选面积小的
        for i in range(len(contours)):
                cnt = contours[i]
                # 计算该轮廓的面积
                area = cv2.contourArea(cnt)

                # 面积小的都筛选掉
                if (area < 2000):
                        continue

                # 轮廓近似，作用很小
                epsilon = 0.001 * cv2.arcLength(cnt,True)
                approx = cv2.approxPolyDP(cnt, epsilon, True)

                # 找到最小的矩形，该矩形可能有方向
                rect = cv2.minAreaRect(cnt)
                print "rect is: "
                print rect

                # box是四个点的坐标
                box = cv2.cv.BoxPoints(rect)
                box = np.int0(box)

                # 计算高和宽
                height = abs(box[0][1] - box[2][1])
                width = abs(box[0][0] - box[2][0])
                # 车牌正常情况下长高比在2.7-5之间
                ratio =float(width) / float(height)
                print ratio
                if (ratio > 5 or ratio < 2):
                        continue
                region.append(box)

        return region

def detect(img):
        # 转化成灰度图
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 形态学变换的预处理
        dilation = preprocess(gray)
        # 查找车牌区域
        region = findPlateNumberRegion(dilation)

        # 用绿线画出这些找到的轮廓
        for box in region:
                cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
        ys = [box[0, 1], box[1, 1], box[2, 1], box[3, 1]]
        xs = [box[0, 0], box[1, 0], box[2, 0], box[3, 0]]
        ys_sorted_index = np.argsort(ys)
        xs_sorted_index = np.argsort(xs)

        x1 = box[xs_sorted_index[0], 0]
        x2 = box[xs_sorted_index[3], 0]

        y1 = box[ys_sorted_index[0], 1]
        y2 = box[ys_sorted_index[3], 1]

        img_org2 = img.copy()
        img_plate = img_org2[y1:y2, x1:x2]

        return img_plate
        # cv2.imshow('number plate', img_plate)
        # cv2.imwrite('number_plate'+str(i)+'.jpg', img_plate)

        # cv2.namedWindow('img', cv2.WINDOW_NORMAL)
        # cv2.imshow('img', img)

        # 带轮廓的图片
        # cv2.imwrite('contours.png', img)

        # cv2.waitKey(0)
        # 关闭窗口
        # cv2.destroyAllWindows()
def mul(img):
        a,b,c=img.shape
        return a*b


if __name__ == '__main__':
        for i in range(1,11):
                image1 = "image/"+str(i)+".jpg"
                rotate(image1,"temp/temp.jpg")
                image2= "temp/temp.jpg"

                img = cv2.imread(image1)
                image11=detect(img)
                # 旋转后的
                img = cv2.imread(image2)
                image12=detect(img)
                a=mul(image11)
                b=mul(image12)
                if a>b:
                        cv2.imwrite('number_plate'+str(i)+'.jpg', image11)
                else:
                        cv2.imwrite('number_plate'+str(i)+'.jpg', image12)
                        rotate('number_plate'+str(i)+'.jpg','number_plate'+str(i)+'.jpg')